In [3]:
suppressMessages({
  library("sleuth")
})

In [4]:
?sleuth_prep

sleuth_prep {sleuth},R Documentation
sample_to_covariates,"a data.frame which contains a mapping from sample (a required column) to some set of experimental conditions or covariates. The column path is also required, which is a character vector where each element points to the corresponding kallisto output directory. The column sample should be in the same order as the corresponding entry in path."
full_model,an R formula which explains the full model (design) of the experiment OR a design matrix. It must be consistent with the data.frame supplied in sample_to_covariates. You can fit multiple covariates by joining them with '+' (see example)
target_mapping,"a data.frame that has at least one column 'target_id' and others that denote the mapping for each target. if it is not NULL, target_mapping is joined with many outputs where it might be useful. For example, you might have columns 'target_id', 'ensembl_gene' and 'entrez_gene' to denote different transcript to gene mappings. Note that sleuth_prep will treat all columns as having the 'character' data type."
aggregation_column,a string of the column name in target_mapping to aggregate targets (typically to summarize the data on the gene level). The aggregation is done using a p-value aggregation method when generating the results table. See sleuth_results for more information.
num_cores,an integer of the number of computer cores mclapply should use to speed up sleuth preparation
...,any of several other arguments that can be used as advanced options for sleuth preparation. See details.


In [7]:
species_tx2gene_map = list("bos_taurus"= "/home//cmb-panasas2/skchoudh/genomes/bos_taurus/annotation/Bos_taurus.UMD3.1.94.tx2gene.tsv",
                       "rattus_norvegicus"="/home/cmb-panasas2/skchoudh/genomes/rattus_norvegicus/annotation/Rattus_norvegicus.Rnor_6.0.94.tx2gene.tsv",
                       "pongo_abelii"="/home/cmb-panasas2/skchoudh/genomes/pongo_abelii/annotation/Pongo_abelii.PPYG2.94.tx2gene.tsv",
                       "monodelphis_domestica"="/home/cmb-panasas2/skchoudh/genomes/monodelphis_domestica/annotation/Monodelphis_domestica.monDom5.94.tx2gene.tsv",
                       "macaca_mulatta"="/home/cmb-panasas2/skchoudh/genomes/macaca_mulatta/annotation/Macaca_mulatta.Mmul_8.0.1.94.tx2gene.tsv",
                       "pan_troglodytes" ="/home/cmb-panasas2/skchoudh/genomes/pan_troglodytes/annotation/Pan_troglodytes.Pan_tro_3.0.94.tx2gene.tsv",
                       "mus_musculus"="/home/cmb-panasas2/skchoudh/genomes/mus_musculus/annotation/Mus_musculus.GRCm38.94.tx2gene.tsv",
                       "homo_sapiens" = "/home/cmb-panasas2/skchoudh/genomes/homo_sapiens/annotation/Homo_sapiens.GRCh38.94.tx2gene.tsv",
                       "gallus_gallus" = "/home/cmb-panasas2/skchoudh/genomes/gallus_gallus//annotation//Gallus_gallus.Gallus_gallus-5.0.94.tx2gene.tsv",
                       "ornithorhynchus_anatinus" = "/home/cmb-panasas2/skchoudh/genomes/ornithorhynchus_anatinus/annotation/Ornithorhynchus_anatinus.OANA5.94.tx2gene.tsv",
                       "gorilla_gorilla" = "/home/cmb-panasas2/skchoudh/genomes/gorilla_gorilla/annotation/Gorilla_gorilla.gorGor4.94.tx2gene.tsv",
                       "pan_paniscus" = "/home/cmb-panasas2/skchoudh/genomes/pan_paniscus/annotation//Pan_paniscus.panpan1.1.94.tx2gene.tsv"
                      )

In [ ]:
collapse_to_gene <- function (species, srp, sampleInfo){
    files <- file.path('/staging/as/skchoudh/rna-seq-output/', species, srp, '/counts/', sampleInfo$sample)
    names(files) <- sampleInfo$tissue
    print(species_tx2gene_map[[species]])
    tx2gene <- read.table(species_tx2gene_map[[species]], header=FALSE)
    txi.kallisto <- sle(files, type = "kallisto", ignoreTxVersion = TRUE, tx2gene = tx2gene)
    df <- as.data.frame(txi.kallisto$counts)
    #df$genes <- rownames(df)
    write.table(df, file=file.path('/home//cmb-panasas2/skchoudh/github_projects/EE-546-project/cross-species-data/', paste0(srp, '-', species, '_kallisto_gene_tables.tsv')),  quote=F, sep='\t', row.names=T, col.names=T)
}


In [20]:
samples <- read.table('../cross-species-data/SRP136499_sample_info.tsv', header = TRUE, sep='\t')
species <- 'pan_troglodytes'
srp <- 'SRP136499'
sampleInfo<- samples[samples$species==species,]

files <- file.path('/staging/as/skchoudh/rna-seq-output', species, srp, 'counts', sampleInfo$sample)
sampleInfo$path <- files
#names(files) <- sampleInfo$tissue
print(species_tx2gene_map[[species]])
t2g <- read.table(species_tx2gene_map[[species]], header=FALSE)
colnames(t2g) <- c('target_id', 'gene_id')

[1] "/home/cmb-panasas2/skchoudh/genomes/pan_troglodytes/annotation/Pan_troglodytes.Pan_tro_3.0.94.tx2gene.tsv"


In [23]:
so <- sleuth_prep(sampleInfo, extra_bootstrap_summary = TRUE, gene_level=TRUE, aggregation_column='gene_id', target_mapping = t2g)


reading in kallisto results
dropping unused factor levels
................


ERROR: Error in check_target_mapping(tmp_names, target_mapping, !is.null(aggregation_column)): couldn't solve nonzero intersection


In [25]:
sampleInfo$path[1]

[1] "/staging/as/skchoudh/rna-seq-output/pan_troglodytes/SRP136499/counts/SRX3849726"

In [24]:
'/staging/as/skchoudh/rna-seq-output/pan_troglodytes/SRP136499/counts/SRX3849726/abundance.h5'

ERROR: Error in eval(expr, envir, enclos): object 'so' not found


In [30]:
test_kal <- sleuth::read_kallisto('/staging/as/skchoudh/rna-seq-output/pan_troglodytes/SRP136499/counts/SRX3849726/abundance.tsv', read_bootstrap = F)
head(test_kal$abundance$target_id)


[1] "ENSPTRT00000000015.4" "ENSPTRT00000000017.4" "ENSPTRT00000000044.4"
[4] "ENSPTRT00000000050.4" "ENSPTRT00000000057.4" "ENSPTRT00000000060.3"

In [29]:
test_kal

	kallisto object

transcripts:  0 
original number of transcripts:  0 
Original or Subset:  Subsetbootstraps:  0 